In [ ]:
import morfeusz2
morf = morfeusz2.Morfeusz()
import pandas as pd
import numpy as np
import re

import string
from string import ascii_lowercase
from sklearn.metrics import f1_score, confusion_matrix

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import unicodedata
!pip install emoji
import emoji
from string import punctuation
import pickle

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install gdown

!gdown --id 1kbYmGkMsQGVBdnDStxx3nz79WNh5JXy7 # training_set_clean_only_text.txt
# !gdown --id 1XP30I7gKxhY1jOCOFlcIWDQDnZ3Qk9rZ # training_set_clean_only_tags.txt
# !gdown --id 1z0Laz8jCGR-GQcMNL2_qKsGnbL4VRRCY # test_set_clean_only_text.txt
# !gdown --id 1o_XRCnKlScCrZRk5suzjwzljT7YonQ2q # test_set_clean_only_tags.txt


!gdown --id 1gJL8GYTuWj5BXUUFnCy--AE4xEZlDr1u # corpora

!gdown --id 1-IeRYiTy5rujiTg97jrEzhCSpdJkoBL1 # dataset4

!gdown --id 1IezKpxY0c8OHv0nq44yrUH8pYtHnsnY- # dataset5


## LSH with simpler processing

In [ ]:
print(gensim.__version__)

3.6.0


In [ ]:
dataset_clean = []
with open('dataset_clean4.txt', 'r') as file2:
  for i, line in enumerate(file2):
    dataset_clean.append(line.split())

In [ ]:
def read_file(file_name): # read given file and create list of phrases
  lines = []
  with open(file_name, encoding="UTF-8-SIG") as text:
    for line in text:
      lines.append(line[:-1])
  return lines
  
text = read_file('training_set_clean_only_text.txt')

In [ ]:
text[:10]

['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
 '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
 '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
 '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
 'Odrzut natychmiastowy, kwaśna mina, mam problem',
 'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
 '@anonymized_account No nie ma u nas szczęścia 😉',
 '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
 '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
 '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account']

In [ ]:
not_wanted_words = ['@anonymized_account']

def word_process(word):
  if word in not_wanted_words:
    return False

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
  word = emoji_pattern.sub(r'', word) # no emoji
  word = word.lower()
  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break 
  word = word.lower()
  if len(word)==0:
    return False
  return word

def clean_dataset(dataset):
  dataset_clean = []
  for line in dataset:
    row = []
    words = line.split()
    if words[0] == 'RT': 
      continue
    for word in words:
      word = word_process(word)
      if word != False: 
        row.append(word)
    dataset_clean.append(' '.join(row))
  return dataset_clean

clean_train = clean_dataset(text)

In [ ]:
clean_train[:10]

['dla ja faworyt do tytuł być cracovia zobaczyć czy typ się sprawdzić',
 'brawo ty daria kibic mieć być na dobry i złe',
 'super polski premier składać kwiat na grób kolaborant ale doczekać czas',
 'muszy inny drogi nie mama',
 'odrzut natychmiastowy kwaśny mina mama problem',
 'jaki on być fajny xdd pamiętać że spóźnić się na on pierwsze zajęcia i to sporo i za kara kazać księga_micheasza usiąść w pierwszy ławka xd',
 'no nie mieć u my szczęście',
 'dawno kto taka wredny nie widzieć xd',
 'zaległość były ale ważny czy były wezwanie do zapłata z który się klub nie wywiązać',
 'gdzie być . brudziński być kłamca i marny kutas']

In [ ]:
for sen in clean_train:
  dataset_clean.append(sen.split())
len(dataset_clean)

5009385

In [ ]:
def cossim(u,v):
  if np.all(u == v):
    return 1
  norm = np.linalg.norm(u)*np.linalg.norm(v)
  cosine = u@v/norm
  ang = np.arccos(cosine)
  return 1-ang/np.pi

In [ ]:
model_embeddings128_fast_with_train = gensim.models.Word2Vec.load("word2vec_fast_128_with_train.model")

In [ ]:
cossim(model_embeddings128_fast_with_train['kobieta'], model_embeddings128_fast_with_train['1'])

/tmp/ipykernel_42608/158147224.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  cossim(model_embeddings128_fast_with_train['kobieta'], model_embeddings128_fast_with_train['1'])


0.5876831846296927

In [ ]:
def create_trainings_dict(train_data):
  training_dict = dict()
  for sen in train_data:
    sen = sen.split()
    for word in sen:
      if word in training_dict:
        training_dict[word] +=1 
      else:
        training_dict[word] = 1

  training_word_min_count2 = []
  for word in training_dict:
    if training_dict[word] >= 2:
      training_word_min_count2.append(word)
  return training_dict, training_word_min_count2

training_dict, training_word_min_count2 = create_trainings_dict(clean_train)

In [ ]:
len(training_dict)

13715

In [ ]:
len(training_word_min_count2)

5934

### train 512 features

In [ ]:
def create_lsh(embeddings, training_dict_words, feature_size=128, threshold=0.61):
  chosen_words = np.random.choice(np.arange(len(training_dict_words)), feature_size)
  embeddings_dict_size = len(embeddings.wv.vocab)
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)

  feature_dim = 0
  words_in_embeddings = set()
  for word in embeddings.wv.vocab:
    words_in_embeddings.add(word)
  for chosen_word_id in chosen_words:

    print(feature_dim)
    curr_word = training_dict_words[chosen_word_id]
    
    for word in words_in_embeddings:
      sim_value = cossim(embeddings[word], embeddings[curr_word])

      if sim_value >= threshold:
        lsh_dict[word][feature_dim] = 1
      else:
        lsh_dict[word][feature_dim] = 0
    feature_dim += 1
  return lsh_dict, chosen_words

lsh_dict1, chosen_words1 = create_lsh(model_embeddings128_fast_with_train, training_word_min_count2, feature_size=512)


In [ ]:
lsh_dict1_file = open("lsh_dict1.pickle", "wb")
pickle.dump(lsh_dict1, lsh_dict1_file)
lsh_dict1_file.close()

In [ ]:
with open('chosen_words1.npy', 'wb') as f:
    np.save(f, chosen_words1)

In [ ]:
chosen_words1 = np.load('chosen_words1.npy',allow_pickle=True)

In [ ]:
len(training_word_min_count2)

5934

In [ ]:
chosen_words1[:10]

array([4837, 1057,  150,  593, 5403, 3743, 1590, 2511, 3111,  744])

### add next 512 features

In [ ]:
np.random.choice(list(set(np.arange(len(training_word_min_count2))) - set(chosen_words1)), 10)

array([ 192, 5414, 3448, 3211,  716, 1863,  310, 3123, 2053, 5292])

In [ ]:
def create_lsh_no_repeat(embeddings, training_dict_words, already_chosen, feature_size=128, threshold=0.61):
  chosen_words = np.random.choice(list(set(np.arange(len(training_dict_words))) - set(already_chosen)), feature_size)
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)
  feature_dim = 0
  words_in_embeddings = set()
  for word in embeddings.wv.vocab:
    words_in_embeddings.add(word)
  for chosen_word_id in chosen_words:
    print(feature_dim)
    curr_word = training_dict_words[chosen_word_id]
    for word in words_in_embeddings:
      sim_value = cossim(embeddings[word], embeddings[curr_word])
      if sim_value >= threshold:
        lsh_dict[word][feature_dim] = 1
      else:
        lsh_dict[word][feature_dim] = 0
    feature_dim += 1
  return lsh_dict, chosen_words

lsh_dict3, chosen_words3 = create_lsh_no_repeat(model_embeddings128_fast_with_train, training_word_min_count2, chosen_words1, feature_size=512)


In [ ]:
lsh_dict3_file = open("lsh_dict3.pickle", "wb")
pickle.dump(lsh_dict3, lsh_dict3_file)
lsh_dict3_file.close()

In [ ]:
with open('chosen_words3.npy', 'wb') as f:
    np.save(f, chosen_words3)

In [ ]:
np.concatenate((lsh_dict3['kobieta'], lsh_dict3['kobieta'])).shape

(1024,)

In [ ]:
lsh_dict1['kobieta']

array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,

### run with vector difference as seperation

In [ ]:
def create_lsh2(embeddings, training_dict_words, feature_size=128, threshold=0.61): # with difference in mind
  chosen_words = []
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)

  words_in_embeddings = set()
  for word in embeddings.wv.vocab:
    words_in_embeddings.add(word)
  for feature_dim in range(feature_size):

    chosen_words_curr = np.random.choice(np.arange(len(training_word_min_count2)), 2)
    print(chosen_words_curr)
    curr_word1 = training_dict_words[chosen_words_curr[0]]
    curr_word2 = training_dict_words[chosen_words_curr[1]]

    while len(curr_word1) < 2 or len(curr_word2) < 2:
      chosen_words_curr = np.random.choice(np.arange(len(training_word_min_count2)), 2)
      print(chosen_words_curr)
      curr_word1 = training_dict_words[chosen_words_curr[0]]
      curr_word2 = training_dict_words[chosen_words_curr[1]]
    chosen_words.append([curr_word1, curr_word2])
    cur_vec_sub = embeddings[curr_word1] - embeddings[curr_word2]
        print(feature_dim)
    
    for word in words_in_embeddings:
      sim_value = cossim(cur_vec_sub, embeddings[word])

      if sim_value >= threshold:
        lsh_dict[word][feature_dim] = 1
      else:
        lsh_dict[word][feature_dim] = 0
  return lsh_dict, chosen_words

lsh_dict2, chosen_words2 = create_lsh2(model_embeddings128_fast_with_train, training_word_min_count2, feature_size=512, threshold=0.47)



In [ ]:
lsh_dict2_file = open("lsh_dict2.pickle", "wb")
pickle.dump(lsh_dict2, lsh_dict2_file)
lsh_dict2_file.close()

In [ ]:
with open('chosen_words2.npy', 'wb') as f:
    np.save(f, chosen_words2)

## create vector of 3 values -1 0 1

In [ ]:
def create_lsh_3value(embeddings, training_dict_words, feature_size=128):
  chosen_words = np.random.choice(np.arange(len(training_dict_words)), feature_size)
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)

  sim_array = []
  feature_dim = 0
  words_in_embeddings = []
  for word in embeddings.wv.vocab:
    words_in_embeddings.append(word)
  for chosen_word_id in chosen_words:

    print(feature_dim)
    curr_word = training_dict_words[chosen_word_id]
    sim_array = []
    for word in words_in_embeddings:
      sim_value = cossim(embeddings[word], embeddings[curr_word])
      sim_array.append(sim_value)
    sorted_sim_array = sorted(sim_array)
    threshold1 = sorted_sim_array[int(len(sorted_sim_array)/3)]
    threshold2 = sorted_sim_array[int(len(sorted_sim_array)/3*2)]
    print(threshold1, threshold2)
    for word, value in zip(words_in_embeddings, sim_array):
      if value < threshold1:
        lsh_dict[word][feature_dim] = -1
      elif value < threshold2:
        lsh_dict[word][feature_dim] = 0
      else:
        lsh_dict[word][feature_dim] = 1
    feature_dim += 1

  return lsh_dict, chosen_words

lsh_dict1, chosen_words1= create_lsh_3value(model_embeddings128_fast_with_train, training_word_min_count2, feature_size=128)


In [ ]:
lsh_dict2_file = open("lsh_dict_3val128.pickle", "wb")
pickle.dump(lsh_dict1, lsh_dict2_file)
lsh_dict2_file.close()

In [ ]:
with open('chosen_words_3val128.npy', 'wb') as f:
    np.save(f, chosen_words1)

In [ ]:
def create_lsh_no_repeat_3value(embeddings, training_dict_words, already_chosen, feature_size=128):
  chosen_words = np.random.choice(list(set(np.arange(len(training_dict_words))) - set(already_chosen)), feature_size)
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)
  sim_array = []
  feature_dim = 0
  words_in_embeddings = []
  for word in embeddings.wv.vocab:
    words_in_embeddings.append(word)
  for chosen_word_id in chosen_words:
    print(feature_dim)
    curr_word = training_dict_words[chosen_word_id]
    sim_array = []
    for word in words_in_embeddings:
      sim_value = cossim(embeddings[word], embeddings[curr_word])
      sim_array.append(sim_value)
    sorted_sim_array = sorted(sim_array)
    threshold1 = sorted_sim_array[int(len(sorted_sim_array)/3)]
    threshold2 = sorted_sim_array[int(len(sorted_sim_array)/3*2)]
    print(threshold1, threshold2)
    for word, value in zip(words_in_embeddings, sim_array):
      if value < threshold1:
        lsh_dict[word][feature_dim] = -1
      elif value < threshold2:
        lsh_dict[word][feature_dim] = 0
      else:
        lsh_dict[word][feature_dim] = 1
    feature_dim += 1

  return lsh_dict, chosen_words

lsh_dict2, chosen_words2= create_lsh_no_repeat_3value(model_embeddings128_fast_with_train, training_word_min_count2, chosen_words1, feature_size=512-128)

lsh_dict1_file = open("lsh_dict_3val400.pickle", "wb")
pickle.dump(lsh_dict2, lsh_dict1_file)
lsh_dict1_file.close()


with open('chosen_words_3val400.npy', 'wb') as f:
    np.save(f, chosen_words1)

In [ ]:
lsh_dict2['kobieta']

array([ 1,  1,  1,  1,  1,  0,  0, -1, -1,  0,  0,  0,  0, -1, -1,  0,  0,
        0,  1,  0,  0, -1, -1,  1,  1,  0,  0, -1,  0,  0,  1,  0,  0, -1,
        0,  1,  0,  0,  1, -1,  0,  1, -1, -1,  0,  0,  0,  0,  1,  1, -1,
        0,  1,  1, -1, -1, -1, -1,  1,  0, -1,  1,  0,  1,  0, -1,  1,  1,
        0,  0,  1,  0,  1,  0, -1,  0,  1, -1,  1, -1,  1, -1,  1, -1,  1,
       -1,  0,  0,  0,  1,  0,  0,  1,  1, -1,  0,  1,  1,  0,  1,  1,  1,
        0,  1,  1,  1,  0,  1,  0, -1,  0,  1,  0, -1,  1,  1,  0, -1, -1,
        0,  0, -1, -1,  0,  0,  1, -1,  1, -1, -1,  1,  0,  1,  1,  0, -1,
        1,  1, -1,  1, -1,  1, -1, -1,  0,  1,  0, -1, -1,  0,  1, -1,  1,
        0, -1,  1,  1,  1,  0,  0,  1,  1,  0,  1,  0, -1,  1, -1,  1,  0,
        0,  1,  0,  1,  1,  1,  1,  0,  1,  1,  0,  0,  1,  0,  1,  0, -1,
        1, -1,  0,  0,  1,  0, -1,  0,  0, -1,  1,  1, -1,  0,  1,  0,  0,
        1,  0,  1,  1,  0,  0,  0,  0,  0,  1, -1,  1,  0,  1,  1, -1,  0,
        1, -1,  0, -1, -1

In [ ]:
lsh_dict1['kobieta'] 

array([ 1,  1, -1, -1,  1,  0,  0, -1,  0,  0,  1,  0,  1,  1,  0, -1,  0,
        0,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1, -1,  1, -1,  0,  0, -1,
        0,  0,  1,  0,  1,  0,  0,  0,  0, -1, -1,  0,  0, -1,  0, -1,  0,
       -1,  1,  0, -1,  0,  1,  1,  0,  1,  0,  0, -1,  1, -1,  0, -1, -1,
       -1,  1,  0,  1, -1,  0,  1, -1,  0, -1, -1,  0,  0,  0,  1,  0, -1,
        0,  0, -1,  0, -1, -1,  1,  1,  1,  0,  0, -1,  0,  1,  0,  1,  0,
        1, -1,  1, -1,  0,  1,  1, -1, -1,  1,  0, -1,  0,  0,  1, -1,  1,
        0, -1,  1,  0,  1,  1, -1, -1,  1], dtype=int16)

In [ ]:
np.concatenate((lsh_dict1['kobieta'], lsh_dict2['kobieta'])).shape

(512,)

In [ ]:
f = open("lsh_dict_3val400.pickle",'rb')
lsh_dict2 = pickle.load(f)
f.close()

In [ ]:
for word in lsh_dict2:
  lsh_dict2[word] = np.concatenate((lsh_dict1[word], lsh_dict2[word]))

In [ ]:
len(lsh_dict2)

990644

In [ ]:
lsh_dict2['kobieta'].shape

(512,)

In [ ]:
lsh_dict1_file = open("lsh_dict_3val512.pickle", "wb")
pickle.dump(lsh_dict2, lsh_dict1_file)
lsh_dict1_file.close()

## LSH with improved processing

In [ ]:
dataset_clean = []
with open('dataset_clean5.txt', 'r') as file2:
  for i, line in enumerate(file2):
    dataset_clean.append(line.split())

In [ ]:
def read_file(file_name): # read given file and create list of phrases
  lines = []
  with open(file_name, encoding="UTF-8-SIG") as text:
    for line in text:
      lines.append(line[:-1])
  return lines
  
text = read_file('training_set_clean_only_text.txt')

In [ ]:
emojis_to_words = {'CONFOUNDED FACE': 'zmieszanie',
                  'CRYING FACE': 'płacz',
                  'DISAPPOINTED BUT RELIEVED FACE': 'rozczarowanie',
                  'DISAPPOINTED FACE': 'rozczarowanie',
                  'EXPRESSIONLESS FACE': 'bezwyrazowy',
                  'FACE SAVOURING DELICIOUS FOOD': 'pyszności',
                  'FACE SCREAMING IN FEAR': 'strach',
                  'FACE THROWING A KISS': 'pocałunek',
                  'FACE WITH COLD SWEAT': 'stres',
                  'FACE WITH MEDICAL MASK': 'maska',
                  'FACE WITH OK GESTURE': 'ok',
                  'FACE WITH OPEN MOUTH': 'zdziwienie',
                  'FACE WITH STUCK-OUT TONGUE': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND WINKING EYE': 'język',
                  'FACE WITH TEARS OF JOY': 'radość',
                  'FEARFUL FACE': 'strach',
                  'FLUSHED FACE': 'zaczerwienienie',
                  'GRIMACING FACE': 'grymas',
                  'GRINNING FACE': 'uśmiech',
                  'GRINNING FACE WITH SMILING EYES': 'uśmiech',
                  'HAPPY PERSON RAISING ONE HAND': 'radość',
                  'HEAR-NO-EVIL MONKEY': 'małpka',
                  'HUSHED FACE': 'uciszenie',
                  'LOUDLY CRYING FACE': 'płacz',
                  'NEUTRAL FACE': 'neutralność',
                  'PENSIVE FACE': 'zamyślenie',
                  'PERSEVERING FACE': 'wytrwałość',
                  'PERSON BOWING DEEPLY': 'ukłon',
                  'PERSON RAISING BOTH HANDS IN CELEBRATION': 'radość',
                  'PERSON WITH FOLDED HANDS': 'ok',
                  'POUTING FACE': 'dąsanie',
                  'SEE-NO-EVIL MONKEY': 'małpka',
                  'SLIGHTLY SMILING FACE': 'uśmiech',
                  'SMILING FACE WITH HALO': 'uśmiech',
                  'SMILING FACE WITH HEART-SHAPED EYES': 'miłość',
                  'SMILING FACE WITH OPEN MOUTH': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND COLD SWEAT': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES': 'uśmiech',
                  'SMILING FACE WITH SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH SUNGLASSES': 'uśmiech',
                  'SPEAK-NO-EVIL MONKEY': 'małpka',
                  'UNAMUSED FACE': 'nierozbawienie',
                  'UPSIDE-DOWN FACE': 'uśmiech',
                  'WEARY FACE': 'zmęczenie',
                  'WINKING FACE': 'mrugnięcie'}

In [ ]:
not_wanted_words = ['@anonymized_account', 'account', 'anonymized' ]

emoji_pattern = re.compile("["
          # u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U0001F700-\U0001F77F"  # alchemical symbols
          u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
          u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
          u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
          u"\U0001FA00-\U0001FA6F"  # Chess Symbols
          u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
          u"\U00002702-\U000027B0"  # Dingbats
          u"\U000024C2-\U0001F251" 
                            "]+", flags=re.UNICODE)

only_faces_emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
                            "]+", flags=re.UNICODE)


def word_process_with_emoji(word, previous_word=''):
  if word in not_wanted_words:
    return False

  if emoji_pattern.match(word):
    return False
  if only_faces_emoji_pattern.match(word):
    word = unicodedata.name(word)
    if word in emojis_to_words: 
      word = emojis_to_words[word]
    else: return False
    if word == previous_word:
      return False
    return word
  word = word.lower()

  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break
  word = word.lower()
  if len(word)==0:
    return False
  return word


def clean_dataset_with_emoji(dataset):
  dataset_clean = []
  re_split = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))
  for line in dataset:
    row = []
    words = re_split.split(line)
    if words[0] == 'RT': 
      continue
    new_words = []
    for word in words:
      if len(word) == 0: continue
      if not emoji_pattern.match(word) and not only_faces_emoji_pattern.match(word):
        new_words.append(word)
      elif len(word) == 1:
        new_words.append(word)
      else:
        word = emoji.get_emoji_regexp().split(word)
        word = [i for i in word if len(i)>0]
        new_words += word

    previous_word = ''
    for word in new_words:
      word = word_process_with_emoji(word, previous_word)
      if word != False:
        previous_word = word
      if word != False: 
        row.append(word)
    dataset_clean.append(' '.join(row))
  return dataset_clean

clean_train = clean_dataset_with_emoji(text)

In [ ]:
for sen in clean_train:
  dataset_clean.append(sen.split())
len(dataset_clean)

5009384

In [ ]:
model_embeddings128_fast_with_train = gensim.models.Word2Vec.load("word2vec_fast_128_with_train__emoji_rerun.model")

In [ ]:
def create_lsh(embeddings, training_dict_words, feature_size=128, threshold=0.61):
  chosen_words = np.random.choice(np.arange(len(training_dict_words)), feature_size)
  lsh_dict = {}
  for word in embeddings.wv.vocab:
    lsh_dict[word] = np.zeros((feature_size,), dtype=np.short)

  feature_dim = 0
  words_in_embeddings = set()
  for word in embeddings.wv.vocab:
    words_in_embeddings.add(word)
  for chosen_word_id in chosen_words:
    print(feature_dim)
    curr_word = training_dict_words[chosen_word_id]
    
    for word in words_in_embeddings:
      sim_value = cossim(embeddings[word], embeddings[curr_word])

      if sim_value >= threshold:
        lsh_dict[word][feature_dim] = 1
      else:
        lsh_dict[word][feature_dim] = 0
    feature_dim += 1

  return lsh_dict, chosen_words


In [ ]:
lsh_dict1, chosen_words1 = create_lsh(model_embeddings128_fast_with_train, training_word_min_count2, feature_size=512)

lsh_dict1_file = open("lsh_dict1_emoji_rerun.pickle", "wb")
pickle.dump(lsh_dict1, lsh_dict1_file)
lsh_dict1_file.close()


with open('chosen_words1_emoji_rerun.npy', 'wb') as f:
    np.save(f, chosen_words1)